In [1]:
! conda install pytorch-nightly -c pytorch --yes

Solving environment: done

## Package Plan ##

  environment location: /Users/abhinavverma/anaconda3

  added / updated specs:
    - pytorch-nightly


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.11               |           py37_0         1.7 MB
    ninja-1.9.0                |   py37h04f5b5a_0          96 KB
    pytorch-nightly-1.0.0.dev20190405|          py3.7_0        47.7 MB  pytorch
    ------------------------------------------------------------
                                           Total:        49.5 MB

The following NEW packages will be INSTALLED:

  ninja              pkgs/main/osx-64::ninja-1.9.0-py37h04f5b5a_0
  pytorch-nightly    pytorch/osx-64::pytorch-nightly-1.0.0.dev20190405-py3.7_0

The following packages will be UPDATED:

  conda                                        4.6.8-py37_0 --> 4.6.11-py37_0



pytorch-nightly-1.0. | 47.7 MB   | ##############

In [1]:
! conda uninstall --force pytorch


PackagesNotFoundError: The following packages are missing from the target environment:
  - pytorch




In [1]:
import torch
from exp.nb_Training import *

In [8]:
from fastai import datasets
from time import sleep

In [21]:
class SlowCalculator():
    def __init__(self, cb=None): self.cb,self.res = cb,0
    
    def __call__(self, cb_name, *args):
        if not self.cb: return
        cb = getattr(self.cb,cb_name, None)
        if cb: return cb(self,*args) #self stands for the instance of SlowCalculator()

    def calc(self):
        for i in range(5):
            self('before_calc', i)
            self.res += i*i
            sleep(1)
            if self('after_calc', i):
                print("stopping early")
                break

In [22]:
class ModifyingCallback():
    def after_calc (self, calc, epoch):
        print(type(calc))
        print(f"After {epoch}: {calc.res}")
        if calc.res>10: return True
        if calc.res<3: calc.res = calc.res*2

In [23]:
calculator = SlowCalculator(ModifyingCallback())

In [24]:
calculator.calc()

<class '__main__.SlowCalculator'>
After 0: 0
<class '__main__.SlowCalculator'>
After 1: 1
<class '__main__.SlowCalculator'>
After 2: 6
<class '__main__.SlowCalculator'>
After 3: 15
stopping early


In [25]:
torch.get_num_threads()

2

In [2]:
from pathlib import Path
import gzip,pickle
from torch import tensor

In [5]:
MNIST_URL = "http://deeplearning.net/data/mnist/mnist.pkl"

In [3]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))
def normalize(x,m,s):
    return (x-m)/s

In [6]:
x_train,y_train,x_valid,y_valid = get_data()

In [7]:
train_mean,train_std = x_train.mean(),x_train.std()

In [8]:
#export
def normalize_to(train, valid):
    m,s = train.mean(),train.std()
    return normalize(train, m, s), normalize(valid, m, s)

In [9]:
x_train,x_valid = normalize_to(x_train,x_valid)

In [10]:
#completely generic function to change the size of the input vector to the size we want
def view_tfm(*size):
    print(*size)
    def _inner(x): return x.view(*((-1,)+size))
    return _inner

In [11]:
inner = view_tfm(1,28,28)

1 28 28


In [50]:
??torch.nn.Module

In [12]:
x_train.shape

torch.Size([50000, 784])

In [13]:
inner(x_train).shape

torch.Size([50000, 1, 28, 28])

### Let's create a simple CNN learner layer fit

In [14]:
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

In [15]:
nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy
bs = 64

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [16]:
#export
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x): return self.func(x)

def flatten(x):      return x.view(x.shape[0], -1)

In [17]:
def mnist_resize(x): return x.view(-1, 1, 28, 28)

In [18]:
def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_resize),
        nn.Conv2d( 1, 8, 5, padding=2,stride=2), nn.ReLU(), #14
        nn.Conv2d( 8,16, 3, padding=1,stride=2), nn.ReLU(), # 7
        nn.Conv2d(16,32, 3, padding=1,stride=2), nn.ReLU(), # 4
        nn.Conv2d(32,32, 3, padding=1,stride=2), nn.ReLU(), # 2
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32,data.c)
    )

In [19]:
model = get_cnn_model(data)

In [20]:
cbfs = [Recorder, partial(AvgStatsCallback,accuracy)]

In [21]:
opt = optim.SGD(model.parameters(), lr=0.4)
learn = Learner(model, opt, loss_func, data)
run = Runner(cb_funcs=cbfs)

In [22]:
%time run.fit(1, learn)

train: [0.402578828125, tensor(0.8687)]
valid: [0.383096044921875, tensor(0.9036)]
CPU times: user 20.2 s, sys: 2.89 s, total: 23.1 s
Wall time: 7.88 s


The Mac is very slow

In [23]:
def conv2d(ni, nf, ks=3, stride=2):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride), nn.ReLU())

In [24]:
#export
class BatchTransformXCallback(Callback):
    _order=2
    def __init__(self, tfm): self.tfm = tfm
    def begin_batch(self): self.run.xb = self.tfm(self.xb)

def view_tfm(*size):
    def _inner(x): return x.view(*((-1,)+size))
    return _inner

In [25]:
mnist_view = view_tfm(1,28,28)
cbfs.append(partial(BatchTransformXCallback, mnist_view))

In [26]:
cbfs

[exp.nb_Training.Recorder,
 functools.partial(<class 'exp.nb_Training.AvgStatsCallback'>, <function accuracy at 0x1a28961268>),
 functools.partial(<class '__main__.BatchTransformXCallback'>, <function view_tfm.<locals>._inner at 0x1a2895b378>)]

In [27]:
nfs = [8,16,32,32]

In [29]:
def get_cnn_layers(data, nfs):
    nfs = [1] + nfs
    return [
        conv2d(nfs[i], nfs[i+1], 5 if i==0 else 3)
        for i in range(len(nfs)-1)
    ] + [nn.AdaptiveAvgPool2d(1), Lambda(flatten), nn.Linear(nfs[-1], data.c)]

def get_cnn_model(data, nfs): return nn.Sequential(*get_cnn_layers(data, nfs))

In [30]:
get_cnn_layers(data,nfs)

[Sequential(
   (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (1): ReLU()
 ), Sequential(
   (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (1): ReLU()
 ), Sequential(
   (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (1): ReLU()
 ), Sequential(
   (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (1): ReLU()
 ), AdaptiveAvgPool2d(output_size=1), Lambda(), Linear(in_features=32, out_features=10, bias=True)]

In [31]:
#export
def get_runner(model, data, lr=0.6, cbs=None, opt_func=None, loss_func = F.cross_entropy):
    if opt_func is None: opt_func = optim.SGD
    opt = opt_func(model.parameters(), lr=lr)
    learn = Learner(model, opt, loss_func, data)
    return learn, Runner(cb_funcs=listify(cbs))

In [32]:
model = get_cnn_model(data, nfs)
learn,run = get_runner(model, data, lr=0.4, cbs=cbfs)

In [33]:
learn1,run1 = get_runner(model, data, lr=0.4, cbs=cbfs)

In [34]:
%time run.fit(3, learn)

train: [0.469774453125, tensor(0.8426)]
valid: [0.1129917724609375, tensor(0.9672)]
train: [0.09288205078125, tensor(0.9717)]
valid: [0.09296600952148437, tensor(0.9713)]
train: [0.0712677978515625, tensor(0.9770)]
valid: [0.3370674560546875, tensor(0.9073)]
CPU times: user 1min, sys: 9.13 s, total: 1min 10s
Wall time: 24.4 s
